In [1]:
%load_ext autoreload
%autoreload 2

import os, json

import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import animation
from datatools.trace_animator import TraceAnimator
from datatools.trace_helper import TraceHelper
from datatools.visualize_helper import VisualizeHelper
from datatools.nba_helper import NBADataHelper, NBADataAnimator
from datatools.nfl_helper import NFLDataHelper

from models import load_model
from models.utils import print_helper, reshape_tensor, get_dataset_config, normalize_tensor

from models.baselines.graphimputer.graphimputer import BidirectionalGraphImputer

## Model evaluating on test data

### Load model

In [2]:
device = "cuda:0"
trial = 3003
save_path = f"saved/{trial:03d}"

with open(f"{save_path}/params.json", "r") as f:
    params = json.load(f)

if params["model"] == "nrtsi":
    model = load_model(params["model"], params).to(device)

    gap_models_dict = dict()

    gap_models_dict[1] = f"{save_path}/model/nrtsi_state_dict_best_gap_1.pt"
    gap_models_dict[2] = f"{save_path}/model/nrtsi_state_dict_best_gap_2.pt"
    gap_models_dict[4] = f"{save_path}/model/nrtsi_state_dict_best_gap_4.pt"
    gap_models_dict[8] = f"{save_path}/model/nrtsi_state_dict_best_gap_8.pt"
    gap_models_dict[16] = f"{save_path}/model/nrtsi_state_dict_best_gap_16.pt"

    for key in gap_models_dict:
        gap_models_dict[key] = torch.load(gap_models_dict[key], map_location=lambda storage, _: storage)
else:
    state_dict = torch.load(
        f"{save_path}/model/{params['model']}_state_dict_best.pt",
        map_location=lambda storage, _: storage,
    )

    model = load_model(params["model"], params).to(device)
    model.load_state_dict(state_dict)

In [3]:
model_name = params["model"]
dataset = params["dataset"]

if model_name == "ours":
    physics_loss = params["physics_loss"]
    train_hybrid = params["train_hybrid"]

statistic_metrics = True

print(f"-Model name : {model_name}")
print(f"-Dataset : {dataset}")
print(f"-Compute statistic_metrics : {statistic_metrics}")

-Model name : ours
-Dataset : basketball
-Compute statistic_metrics : True


In [4]:
model_keys = ["pred"]
ret_keys = ["n_frames", "n_missings"]
if model.params["model"] == "ours":
    if model.params["physics_loss"]:
        model_keys += ["physics_f", "physics_b"]
    if model.params["train_hybrid"]:
        model_keys += ["static_hybrid", "static_hybrid2", "train_hybrid"]
if statistic_metrics:
    model_keys += ["linear", "knn", "forward"]

    metrics = ["speed", "change_of_step_size", "path_length"]
    ret_keys += [f"{m}_{metric}" for m in model_keys for metric in metrics]

ret_keys += [f"{m}_dist" for m in model_keys]
total_ret = {key: 0 for key in ret_keys}

In [5]:
metrica_files = ["match1.csv", "match2.csv", "match3_valid.csv", "match3_test.csv"]
metrica_paths = [f"data/metrica_traces/{f}" for f in metrica_files]

nba_files = os.listdir("data/nba_traces")
nba_paths = [f"data/nba_traces/{f}" for f in nba_files]
nba_paths.sort()

nfl_files = os.listdir("data/nfl_traces")
nfl_paths = [f"data/nfl_traces/{f}" for f in nfl_files if f.endswith(".csv")]
nfl_paths.sort()

if dataset == "soccer":
    trace_helper = TraceHelper
    test_data_paths = metrica_paths[3:4]
elif dataset == "basketball":
    trace_helper = NBADataHelper
    test_data_paths = nba_paths[90:]
else: # e.g. "American football"
    trace_helper = NFLDataHelper
    test_data_paths = nfl_paths[0:1]

print(f"Test data paths : {test_data_paths}")

Test data paths : ['data/nba_traces/match0021500095.csv', 'data/nba_traces/match0021500096.csv', 'data/nba_traces/match0021500097.csv', 'data/nba_traces/match0021500098.csv', 'data/nba_traces/match0021500099.csv', 'data/nba_traces/match0021500101.csv', 'data/nba_traces/match0021500102.csv', 'data/nba_traces/match0021500103.csv', 'data/nba_traces/match0021500104.csv', 'data/nba_traces/match0021500105.csv']


### Run model

In [6]:
for path in test_data_paths:
    print()
    print(path,":")
    match_traces = pd.read_csv(path, header=0, encoding="utf-8-sig")
    helper = trace_helper(traces=match_traces)

    if model_name == "nrtsi":
        ret, df_dict = helper.predict(model, gap_models=gap_models_dict, statistic_metrics=statistic_metrics, dataset=dataset)
    else:
        ret, df_dict = helper.predict(model, statistic_metrics=statistic_metrics, dataset=dataset)

    for key, value in ret.items():
        total_ret[key] += value
        
    print()

print("Total Performance:")
print_helper(total_ret, model_keys, trial=trial, save_txt=True)

torch.save(helper, f"{save_path}/helper")
torch.save(df_dict, f"{save_path}/df_dict")


data/nba_traces/match0021500095.csv :


Episode: 100%|██████████| 65/65 [00:06<00:00,  9.31it/s]




data/nba_traces/match0021500096.csv :


Episode: 100%|██████████| 72/72 [00:06<00:00, 10.66it/s]




data/nba_traces/match0021500097.csv :


...



data/nba_traces/match0021500104.csv :


Episode: 100%|██████████| 72/72 [00:06<00:00, 11.38it/s]




data/nba_traces/match0021500105.csv :


Episode: 100%|██████████| 73/73 [00:06<00:00, 11.40it/s]



Total Performance:
pred_speed : 0.7269081
pred_change_of_step_size : 7.44e-06
pred_path_length : 0.00225891
pred_dist : 1.69324348
physics_f_speed : 0.64902775
physics_f_change_of_step_size : 7.17e-06
physics_f_path_length : 0.00242646
physics_f_dist : 1.69696024
physics_b_speed : 0.6499614
physics_b_change_of_step_size : 7.34e-06
physics_b_path_length : 0.00239357
physics_b_dist : 1.70962391
static_hybrid_speed : 0.6327552
static_hybrid_change_of_step_size : 6.81e-06
static_hybrid_path_length : 0.00240879
static_hybrid2_speed : 0.62020023
static_hybrid2_change_of_step_size : 6.99e-06
static_hybrid2_path_length : 0.00248064
static_hybrid_dist : 1.64373574
static_hybrid2_dist : 1.63704332
train_hybrid_speed : 0.61762303
train_hybrid_change_of_step_size : 6.93e-06
train_hybrid_path_length : 0.00245594
train_hybrid_dist : 1.63327142
linear_speed : 1.15886676
linear_change_of_step_size : 2.021e-05
linear_path_length : 0.00573541
linear_dist : 4.13439601
knn_speed : 3.0950726
knn_change_of

## Performance analysis

##### (1) Get Main model results

In [7]:
trial = 3003
save_path = f"saved/{trial:03d}"
if os.path.isfile(save_path + "/df_dict"):
    helper =  torch.load(save_path + "/helper")
    df_dict = torch.load(save_path + "/df_dict")
    with open(f"{save_path}/params.json", "r") as f:
        params = json.load(f)

##### (2) Add baseline model results

In [8]:
# trial_dict = {4000 : "brits", 5000 : "naomi", 214 : "nrtsi"} # Metrica
trial_dict = {4003 : "brits", 5001 : "naomi", 6001 : "nrtsi", 9996 : "graphimputer"} # NBA
for (t, model_name) in trial_dict.items():
    save_path = f"saved/{t:03d}"
    if os.path.isfile(save_path + "/df_dict"):
        df_dict_ = torch.load(save_path + "/df_dict")
        df_dict[f"{model_name}_df"] = df_dict_["pred_df"]

In [9]:
df_dict.keys()

dict_keys(['target_df', 'mask_df', 'pred_df', 'physics_f_df', 'physics_b_df', 'static_hybrid_df', 'static_hybrid2_df', 'train_hybrid_df', 'linear_df', 'knn_df', 'forward_df', 'train_hybrid_weights_df', 'brits_df', 'naomi_df', 'nrtsi_df', 'graphimputer_df'])

### Animation

##### (1) Soccer Animator

In [10]:
helper.traces["episode"].unique()

array([  1,   2,   0,   4,   6,   8,  16,  18,  20,  22,  24,  26,  28,
        30,  32,  36,  38,  41,  43,  44,  46,  48,  51,  53,  54,  55,
        57,  58,  62,  63,  64,  66,  68,  70,  76,  80,  83,  85,  86,
        88,  90,  92,  94,  95,  96,  97,  99, 101, 103, 105, 107, 111,
       112, 114, 118, 120, 121, 123, 124, 126, 127, 129, 131, 133, 136,
       138, 140, 142, 144, 145, 146, 148, 152, 154])

In [ ]:
i0 = 479
i1 = 873

animator = TraceAnimator(
    trace_dict={"main": df_dict["target_df"][i0:i1], "pred": df_dict["train_hybrid_df"][i0:i1]},
    mask = df_dict["mask_df"][i0:i1],
    show_episodes=True,
    show_events=False,
    show_frames=False,
    show_polygon=True,
    annot_cols=None,
)
anim = animator.run()

path = f"animations/trial_{trial}.mp4"

writer = animation.FFMpegWriter(fps=10)
anim.save(path, writer=writer)

##### (2) Basketball Animator

In [ ]:
i0 = 326
i1 = 737
animator = NBADataAnimator(
    trace_dict={"main": df_dict["target_df"][i0:i1], "pred": df_dict["train_hybrid_df"][i0:i1]},
    show_episodes=True,
    show_frames=True,
    masks = df_dict["mask_df"][i0:i1],
)
anim = animator.run()

path = f"animations/trial_{trial}.mp4"

writer = animation.FFMpegWriter(fps=10)
anim.save(path, writer=writer)

### plotting

In [11]:
plot_mode = "imputed_traj" # "imputed_traj", "dist_heatmap", "weights_heatmap"
dataset = params["dataset"]
visualizer = VisualizeHelper(trial, df_dict, plot_mode, dataset=dataset, helper=helper)
visualizer.valid_episodes()

Valid episode idxs : [4, 6, 8, 16, 20, 22, 24, 26, 28, 30, 32, 36, 38, 41, 44, 48, 53, 54, 55, 57, 58, 62, 63, 64, 66, 70, 83, 85, 88, 94, 96, 99, 103, 111, 114, 118, 123, 124, 126, 127, 131, 136, 138, 140, 142, 144, 145, 154]


In [13]:
visualizer.plot_run(epi_idx=0)
plt.close()

Plotting episode : 4
Plot Mode : imputed_traj
Saved path : plots/basketball/3003/imputed_traj/episode_4
